In [ ]:
import torch
import transformers
from torch import cuda, bfloat16


device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda:0')
torch.cuda.empty_cache()

In [ ]:
! pip install -U "transformers==4.38.0" --upgrade

In [ ]:
model_name = 'Kastanie99/gemma-1.1-7b-it-haoran-mt-09052024'

model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast = False)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.eval()
model.to(device)
print(f"Model loaded on {device}")

In [ ]:
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [[tokenizer.eos_token], ["<", "|", "user"]]
]

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
pipe = transformers.pipeline(
    model = model,
    tokenizer = tokenizer,
    return_full_text = True, # Set it to True when combining with LangChain
    task = 'text-generation',
    device = device,
    stopping_criteria=stopping_criteria,  
    temperature = 0.2,
    top_p = 0.15,  
    top_k=0,  
    max_new_tokens = 512,  
    repetition_penalty = 1.3
)

In [ ]:
import datasets

evaluation_set = datasets.load_from_disk("/pfs/data5/home/st/st_us-051500/st_st180358/gemma_training/my_gemma_combined_dataset_test_09052024")
evaluation_set

In [ ]:
evaluation_set[1]

In [ ]:
def extract_fields(example):

    query_start = example['text'].find('<|user|>') + 10  # 找到 '#' 的索引，并向后移动一位
    query_end = example['text'].find('<|assistant|>')  # 找到 '%' 的索引
    query_part = example['text'][query_start:query_end]  # 切片获取两个索引之间的字符串

    completion_start = example['text'].find('<|assistant|>') + 15
    completion_part = example['text'][completion_start:]

    input_end = example['text'].find('<|assistant|>')
    input_for_gemma = example['text'][:input_end] + '<|assistant|>\n'
    
    # 从文本中分割出所需内容
    # parts = example['text'].split('"\n\n')
    # query_part = parts[0].split('"')[-1].strip()
    # completion_part = parts[1].split('Rewritten requirement:\n"')[-1].strip().rstrip('"')
    
    # 返回新的字段
    return {'query': query_part, 'completion': completion_part, 'input_for_gemma': input_for_gemma}

# 应用map函数
updated_dataset = evaluation_set.map(extract_fields)

In [ ]:
updated_dataset

In [ ]:

print(updated_dataset[1]['input_for_gemma'])    
print("``````````````````````")
print(updated_dataset[1]['query'])
print("``````````````````````")
print(updated_dataset[1]['completion'])

In [ ]:
updated_dataset.save_to_disk('/pfs/data5/home/st/st_us-051500/st_st180358/gemma_training/my_gemma_before_prediction_09052024')

In [ ]:
from tqdm import tqdm

completion = []

# instruction = "You are a professional requirements engineer who helps users brainstorm more software requirements."

for input_for_gemma in tqdm(updated_dataset['input_for_gemma']):
    # result = pipe(f"<|system|>\n{instruction}\n<|user|>\n{query}\n<|assistant|>\n")
    result = pipe(input_for_gemma)
    result = result[0]['generated_text'].split('\n<|assistant|>\n')[-1]
    # result = result.strip(" ")
    completion.append(result)

In [ ]:
updated_dataset = updated_dataset.add_column("gemma_preds", completion)

In [ ]:
updated_dataset.save_to_disk("/pfs/data5/home/st/st_us-051500/st_st180358/gemma_training/my_gemma_after_prediction_09052024")

In [ ]:
print(updated_dataset)
print("```````````````````````")
print(updated_dataset[25]['query'])
print("```````````````````````")
print(updated_dataset[25]['gemma_preds'])
print("```````````````````````")
print(updated_dataset[25]['completion'])